# Time Series Project

En la primera etapa nos ocupamos de cargar los datos

# CARGANDO EL DATASET

In [1]:
import pandas as pd # codding
import pandas # for annotations

import warnings

warnings.filterwarnings("ignore")

from paquete_proyecto.iniciando.bases import importar_databases
from paquete_proyecto.herramientas.data_info import data_info

In [2]:
ventas, ventas_sin_duplicados = importar_databases()

La longitud de la serie Nº 1 es de: 127255
La longitud de la serie Nº 2 es de: 108794


In [3]:
display(ventas.sample(3))
data_info(ventas, "ventas")

,IdCliente,NombreCliente,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area
Fecha,,,,,,,,,,,,
2017-06-03,C80134347,CASTILLO JUAN GABRIEL,12.0,SE-CS-2-0001,BALANCEO AUTO,106.0,SERVICIOS,1,6723,Usaquen,Santa ana,33.0
2019-08-30,C860040576,COLTANQUES SAS,24.0,SE-GF-1-0001,ALINEACION,106.0,SERVICIOS,1,20034,Bogota,Grandes flotas,21.0
2016-09-11,C52019992,VEGA RODRIGUEZ ALICIA,10.0,VFR-0018,FILTRO DE AIRE AIP-667,109.0,FILTROS,1,11207,Engativa,Calle 80,31.0


info de ventas,columna,type,count,NaN,NaN_pct,unique,unique_pct
0,IdCliente,object,127255,0,0.0 %,24122,18.96 %
1,NombreCliente,object,127255,0,0.0 %,21840,17.16 %
2,Empleado,float64,127255,0,0.0 %,56,0.04 %
3,Referencia,object,127255,0,0.0 %,1854,1.46 %
4,Descripcion,object,127255,0,0.0 %,1814,1.43 %
5,CodigoFamilia,float64,127255,0,0.0 %,5,0.0 %
6,Familia,object,127255,0,0.0 %,5,0.0 %
7,Cantidad,object,127255,0,0.0 %,213,0.17 %
8,Ventas,object,127255,0,0.0 %,14279,11.22 %
9,Localidad,object,127255,0,0.0 %,6,0.0 %


<div>
    <h4>Evaluación de cada columna</h4>
    <ol>
        <li> <b>IdCliente</b>: object - código alfanumérico</li><br />
        <li> <b>NombreCliente</b>: object - string = Nombre, Apellido</li><br />
        <li> <b>>> Empleado</b>: float - cambiar a int</li><br />
        <li> <b>Referencia</b>: object: categoría ordinal</li><br />
        <li> <b>Descripcion</b>: object: categoria cualitativa </li><br />
        <li> <b>>> CodigoFamilia</b>: float64: Cambiar a int</li><br />
        <li> <b>Familia</b>: object: categoria cualitativa</li><br />
        <li> <b>>>Cantidad</b>: object: cambiar a int</li><br />
        <li> <b>>> Ventas</b>: int: cambiar a float</li><br />
        <li> <b>Localidad</b>: object: categoria bussines</li><br />
        <li> <b>>> Area</b>: float: cambiar a int</li><br />
    </ol>
</div>

In [4]:
from paquete_proyecto.iniciando.variables import ajustar_tipos

In [5]:
data, ventas_sin_duplicados = ajustar_tipos(ventas), ajustar_tipos(ventas_sin_duplicados)

# CREANDO LAS REPRESENTACIONES TEMPORALES
Para cada una de las variables, debería haber una representación temporal, en el que cada registro represente una día unicamente

In [6]:
# crear una pivot table. que en el indice tenga fechas, en las columnas tenga localidad, y que los valores sean la cantidad de clientes que tuvo.

pd.pivot_table(data, values="IdCliente", index="Fecha", columns="Localidad", aggfunc="count")

Localidad,Barrios unidos,Bogota,Engativa,Puente aranda,Suba,Usaquen
Fecha,,,,,,
2016-01-09,7.0,16.0,68.0,11.0,19.0,11.0
2016-01-10,6.0,19.0,37.0,4.0,22.0,8.0
2016-01-11,7.0,17.0,100.0,29.0,13.0,5.0
2016-01-12,5.0,14.0,107.0,5.0,17.0,12.0
2016-02-09,3.0,1.0,46.0,33.0,11.0,7.0
...,...,...,...,...,...,...
2020-10-02,5.0,17.0,79.0,11.0,13.0,NaN
2020-10-03,3.0,NaN,41.0,20.0,16.0,8.0
2020-11-01,2.0,2.0,49.0,12.0,14.0,9.0


In [7]:
# crear una pivot table como la anterior, que en los values tenga cantidad de empleados
pd.pivot_table(data, values="Empleado", index="Fecha", columns="Localidad", aggfunc="count")

print ('Al parecer por cada transacción hay un empleado, no hay 2 transacciones realizadas por el mismo empleado en 1 dia')

Al parecer por cada transacción hay un empleado, no hay 2 transacciones realizadas por el mismo empleado en 1 dia


In [8]:
data.Familia.unique()

array(['LLANTA', 'SERVICIOS', 'REENCAUCHE', 'LUBRICANTES', 'FILTROS'],
      dtype=object)

# COMPLETANDO EL DATASET

Primero tenemos que armar cada una de las series temporales

Para poder aplicar Cross-Validation a la serie temporal, es necesario completar con las fechas faltantes con algun valor default en sus variables. De esta forma podremos garantizar que nuestro folds representan el mismo timedelta.

In [9]:
def date_range(data: pandas.DataFrame) -> pandas.DatetimeIndex:
    """ data must have DatetimeIndex in index"""
    start_date = data.index.min()
    end_date = data.index.max()
    return pd.date_range(start=start_date, end=end_date)

def build_default_sample(index: pandas.DatetimeIndex, index_name: str) -> pandas.DataFrame:
    date_range = index
    muestra = pd.DataFrame([0.0 for i in range(len(date_range))], index=date_range, columns=['default'])
    muestra.index.name = index_name
    return muestra


def complete_dates(data: pandas.DataFrame) -> pandas.DataFrame:
    dates = date_range(data)
    muestra = build_default_sample(dates, data.index.name)
    result = pd.merge(data.reset_index(), muestra.reset_index(), on=data.index.name, how="right")
    result.set_index(data.index.name, inplace=True)
    return result.drop(columns='default')

In [10]:
data = complete_dates(ventas)

# ARMANDO MUESTREOS TEMPORALES

Vamos a hacer cross validation con scikit learn sobre un DataFrame de prueba

In [11]:
def timeseries_cv(data: pandas.DataFrame, n_splits: int, max_train_size: int, test_size: int) -> list[pandas.DataFrame]:
    from sklearn.model_selection import TimeSeriesSplit
    splitter = TimeSeriesSplit(n_splits=n_splits, max_train_size=max_train_size, test_size=test_size)
    date_range = pd.date_range(start = data.index.min(), end= data.index.max())
    train = []
    test = [] #only purpouse is to extract last element
    for train_index, test_index in splitter.split(date_range):
        concat_index, predict_index = data.index[train_index], data.index[test_index]
        train.append(data.loc[concat_index])
        test.append(data.loc[predict_index])
    # Append last element from test into train list
    train.append(test[-1])
    return train

In [12]:
lista_prueba = timeseries_cv(data, 29, 60, 60)

In [13]:
# Son 1790 / 30 = 59,66 días por fold
# k = 29
# splitter = TimeSeriesSplit(n_splits=k)


# Obteniendo muestras bimensuales

Por más que no hay unido los valores díarios en un solo valor por día, voy a intentar aislar todas las fechas

In [14]:
# seleccion de fechas
datasets_bimensuales = []

for indice in range(len(train)):
    fechas = train[indice].index 
    datasets_bimensuales.append(ventas_2.loc[fechas].dropna()) # hay fechas que no están en el index

longitud = []
for elemento in datasets_bimensuales:
    # print (len(elemento))
    longitud.append(len(elemento))

pd.Series(longitud).plot.bar().set(title='Cantidad de ventas bimensuales')

NameError: name 'train' is not defined

In [ ]:
ventas_2

,IdCliente,NombreCliente,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area,default
Fecha,,,,,,,,,,,,,
2016-01-09,C1070956433,PINTO JOHN,24.0,LL-2R-0038,PILOT STREET 110/80-17,101.0,LLANTA,1.0,129310.0,Puente aranda,Calle 13,35.0,0.0
2016-01-09,C19347046,RUIZ LUIS FERNANDO,10.0,SE-CS-2-0001,BALANCEO AUTO,106.0,SERVICIOS,4.0,31035.0,Engativa,Calle 80,31.0,0.0
2016-01-09,C800232717,CORSO Y CIA S EN C,10.0,SE-CS-2-0002,BALANCEO CAMIONETA,106.0,SERVICIOS,4.0,31034.0,Engativa,Calle 80,31.0,0.0
2016-01-09,C79523685,GONZALEZ JUAN CARLOS,10.0,SE-CS-2-0002,BALANCEO CAMIONETA,106.0,SERVICIOS,4.0,31034.0,Engativa,Calle 80,31.0,0.0
2016-01-09,C800214037,EXPRESER SAS,5.0,REB-0063,REENC XZE1 235/75R17.5,102.0,REENCAUCHE,1.0,29892.0,Engativa,Ventas externas,22.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-02,C17050457,GAMBOA PAEZ JOSE MARIO,44.0,SE-CS-1-0001,ALINEACION AUTO,106.0,SERVICIOS,1.0,37815.0,Engativa,Calle 80,31.0,0.0
2020-12-02,C51590721,DAZA MIRIAM,44.0,SE-CS-1-0006,ALINEACION BUS,106.0,SERVICIOS,1.0,37815.0,Engativa,Calle 80,31.0,0.0
2020-12-02,C860517112,TRANSPORTES ESPECIALES COLEGIOS Y TURISMO SA,50.0,SE-CS-1-0005,ALINEACION BUSETA O CAMION LIVIANO,106.0,SERVICIOS,1.0,37815.0,Puente aranda,Calle 13,35.0,0.0


In [ ]:
sedes = list(ventas_2["Sede"].dropna().unique())

for sede in sedes:
    display (ventas_2[ventas_2["Sede"]==sede].tail(1))

,IdCliente,NombreCliente,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area,default
Fecha,,,,,,,,,,,,,
2020-12-02,C860517112,TRANSPORTES ESPECIALES COLEGIOS Y TURISMO SA,50.0,SE-CS-2-0004,BALANCEO BUSETA,106.0,SERVICIOS,2.0,26891.0,Puente aranda,Calle 13,35.0,0.0


,IdCliente,NombreCliente,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area,default
Fecha,,,,,,,,,,,,,
2020-12-02,C51590721,DAZA MIRIAM,44.0,SE-CS-1-0006,ALINEACION BUS,106.0,SERVICIOS,1.0,37815.0,Engativa,Calle 80,31.0,0.0


,IdCliente,NombreCliente,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area,default
Fecha,,,,,,,,,,,,,
2020-12-02,C900315569,VOLVO GROUP COLOMBIA SAS,29.0,SE-CT-1-0003,ALINEACION 3 EJES / CT,106.0,SERVICIOS,1.0,120000.0,Engativa,Ventas externas,22.0,0.0


,IdCliente,NombreCliente,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area,default
Fecha,,,,,,,,,,,,,
2020-12-02,C900349524,ASESORES DE SEGUROS & FIANZAS LTDA,67.0,SE-CS-2-0002,BALANCEO CAMIONETA,106.0,SERVICIOS,1.0,33613.0,Usaquen,Santa ana,33.0,0.0


,IdCliente,NombreCliente,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area,default
Fecha,,,,,,,,,,,,,
2020-12-02,C4266509,GOMEZ LIBARDO,14.0,VFA-0005,FILTRO DE ACEITE FM-1041,109.0,FILTROS,1.0,8403.0,Suba,Suba,32.0,0.0


,IdCliente,NombreCliente,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area,default
Fecha,,,,,,,,,,,,,
2020-12-02,C860002127,INDUSTRIA COLOMBIANA DE LLANTAS SA,2.0,SE-CS-2-0001,BALANCEO AUTO,106.0,SERVICIOS,4.0,33613.0,Barrios unidos,Av Chile,34.0,0.0


,IdCliente,NombreCliente,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area,default
Fecha,,,,,,,,,,,,,
2020-12-02,C800035276,TRANSPORTES MONTEJO SAS,2.0,REC-0088,REENC XDY+ 12.00R24,102.0,REENCAUCHE,1.0,900.0,Bogota,Grandes flotas,21.0,0.0
